In [1]:
# Library import

import pandas as pd
import numpy as np
import random
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, GridSearchCV

random_seed = 123
random.seed(random_seed)
np.random.seed(random_seed)

In [ ]:
def bootstrap(y_true, y_score, iters):
    orig_auc = roc_auc_score(y_true, y_score)
    y_true0_ind = list(np.where(y_true == 0)[0])
    y_true1_ind = list(np.where(y_true == 1)[0])
    samp_size = len(y_score)
    estimated_auc = []
    for i in range(iters):
        if samp_size < 100:
            samp_indices = random.choices(y_true0_ind, k = len(y_true0_ind))
            samp_indices.extend(random.choices(y_true1_ind, k = len(y_true1_ind)))
        else:
            samp_indices = random.choices(range(0, samp_size), k = samp_size)
        samp_score = [y_score[i] for i in samp_indices]
        samp_true = [y_true[i] for i in samp_indices]
        estimated_auc.append(roc_auc_score(samp_true, samp_score) - orig_auc)
    lower = np.percentile(estimated_auc, 2.5)
    upper = np.percentile(estimated_auc, 97.5)
    return (orig_auc + lower, orig_auc + upper)

In [ ]:
def bootstrap_pr(y_true, y_score, iters):
    precision, recall, thresholds = precision_recall_curve(y_true, y_score)
    orig_auc = auc(recall, precision)
    y_true0_ind = list(np.where(y_true == 0)[0])
    y_true1_ind = list(np.where(y_true == 1)[0])
    samp_size = len(y_score)
    estimated_auc = []
    for i in range(iters):
        if samp_size < 100:
            samp_indices = random.choices(y_true0_ind, k = len(y_true0_ind))
            samp_indices.extend(random.choices(y_true1_ind, k = len(y_true1_ind)))
        else:
            samp_indices = random.choices(range(0, samp_size), k = samp_size)
        samp_score = [y_score[i] for i in samp_indices]
        samp_true = [y_true[i] for i in samp_indices]
        precision, recall, thresholds = precision_recall_curve(samp_true, samp_score)
        auc_precision_recall = auc(recall, precision)
        estimated_auc.append(auc_precision_recall - orig_auc)
    lower = np.percentile(estimated_auc, 2.5)
    upper = np.percentile(estimated_auc, 97.5)
    return (orig_auc + lower, orig_auc + upper)

In [2]:
# Loading data
X_train = pd.read_csv("../Data/Split_Data/X_train.csv")
X_train_PCA = pd.read_csv("../Data/Split_Data/X_train_PCA.csv")
y_train = pd.read_csv("../Data/Split_Data/y_train.csv")
X_valid = pd.read_csv("../Data/Split_Data/X_valid.csv")
X_valid_PCA = pd.read_csv("../Data/Split_Data/X_valid_PCA.csv")
y_valid = pd.read_csv("../Data/Split_Data/y_valid.csv")
X_test = pd.read_csv("../Data/Split_Data/X_test.csv")
X_test_PCA = pd.read_csv("../Data/Split_Data/X_test_PCA.csv")
y_test = pd.read_csv("../Data/Split_Data/y_test.csv")

In [3]:
# Training dimensions
X_train_new = pd.concat([X_train, X_valid])
y_train_new = pd.concat([y_train, y_valid])
print(X_train_new.shape)
print(y_train_new.shape)

(1136, 1124)
(1136, 1)


In [4]:
# Training PCA dimensions
X_train_PCA_new = pd.concat([X_train_PCA, X_valid_PCA])
print(X_train_PCA_new.shape)

(1136, 75)


In [5]:
# Model test function
def test(models, X_train, y_train, cv = 5):
    results = {}
    kf = KFold(n_splits=cv)
    for i in models:
        predicted_vals = []
        actual_vals = []
        for j, (train_index, test_index) in enumerate(kf.split(X_train)):
            train_x = X_train.iloc[train_index,:]
            train_y = y_train.iloc[train_index,:]
            test_x = X_train.iloc[test_index,:]
            test_y = y_train.iloc[test_index,:]
            predicted_vals.extend(models[i].fit(train_x, train_y.values.ravel()).predict(test_x))
            actual_vals.extend(test_y.values.ravel())
        results[i] = [roc_auc_score(actual_vals, predicted_vals)]
    return pd.DataFrame(results)

In [6]:
# Full dataset models
reg_params = {'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}

models = {'OLS': LogisticRegression(solver='saga', penalty = "none"),
           'Lasso': GridSearchCV(LogisticRegression(solver='saga', penalty = "l1"), 
                               param_grid=reg_params,
                               cv = 10,
                               scoring = "neg_log_loss").fit(X_train_new, y_train_new.values.ravel()).best_estimator_,
           'Ridge': GridSearchCV(LogisticRegression(solver='saga', penalty = "l2"), 
                               param_grid=reg_params,
                               cv = 10,
                               scoring = "neg_log_loss").fit(X_train_new, y_train_new.values.ravel()).best_estimator_,}

c:\Users\Caitlyn\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Caitlyn\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Caitlyn\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Caitlyn\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Caitlyn\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which 

In [7]:
# LASSO models
reg_params = {'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}

PCA_models = {'OLS': LogisticRegression(solver='saga', penalty = "none"),
           'Lasso': GridSearchCV(LogisticRegression(solver='saga', penalty = "l1"), 
                               param_grid=reg_params,
                               cv = 10,
                               scoring = "neg_log_loss").fit(X_train_PCA_new, y_train_new.values.ravel()).best_estimator_,
           'Ridge': GridSearchCV(LogisticRegression(solver='saga', penalty = "l2"), 
                               param_grid=reg_params,
                               cv = 10,
                               scoring = "neg_log_loss").fit(X_train_PCA_new, y_train_new.values.ravel()).best_estimator_,}

c:\Users\Caitlyn\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Caitlyn\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Caitlyn\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Caitlyn\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Caitlyn\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which 

In [30]:
models

{'OLS': LogisticRegression(penalty='none', solver='saga'),
 'Lasso': LogisticRegression(C=0.1, penalty='l1', solver='saga'),
 'Ridge': LogisticRegression(C=0.01, solver='saga')}

In [8]:
# Full dataset cv
test(models, X_train_new, y_train_new)

c:\Users\Caitlyn\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
c:\Users\Caitlyn\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Caitlyn\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
c:\Users\Caitlyn\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Caitlyn\AppData\Local\Programs\Python\P

,OLS,Lasso,Ridge
0,0.725503,0.799881,0.738723


In [9]:
# PCA models cv
test(PCA_models, X_train_PCA_new, y_train_new)

c:\Users\Caitlyn\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
c:\Users\Caitlyn\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Caitlyn\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
c:\Users\Caitlyn\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Caitlyn\AppData\Local\Programs\Python\P

,OLS,Lasso,Ridge
0,0.800522,0.800522,0.800522


In [10]:
# Full dataset best AUC
full_model = models['Lasso'].fit(X_train_new, y_train_new.values.ravel())

c:\Users\Caitlyn\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [11]:
features = X_train_new.columns
model_coefs = pd.DataFrame(full_model.coef_)
model_coefs.columns = features
model_coefs_abs = abs(model_coefs)
sorted_coefs = model_coefs.loc[:, model_coefs_abs.max().sort_values(ascending=False).index]

beta_coefs = sorted_coefs.values
feature_coefs = sorted_coefs.columns

In [12]:
beta_features = pd.DataFrame({'Feature': list(feature_coefs), 'Beta': list(beta_coefs)[0]})
beta_vals = beta_features.query('abs(Beta) > 0')

In [15]:
pred_y = full_model.predict_proba(X_test)

In [16]:
acc_test = roc_auc_score(y_test, pred_y[:,1])
[lower, upper] = bootstrap(y_test.values.ravel(), pred_y[:,1], 1000)
print("Testing set 95%% AUC CI for the full LASSO model: %5.3f [%5.3f, %5.3f]" % (acc_test, lower, upper))

Testing set 95% AUC CI for the full LASSO model: 0.847 [0.774, 0.909]


In [17]:
# Precision-Recall
precision, recall, thresholds = precision_recall_curve(y_test.values.ravel(), pred_y[:,1])
# AUPRC
auc_precision_recall = auc(recall, precision)
[lower, upper] = bootstrap_pr(y_test.values.ravel(), pred_y[:,1], 1000)
print()
print("Testing set 95%% AUPRC CI for the full LASSO model: %5.3f [%5.3f, %5.3f]" % (auc_precision_recall, lower, upper))


Testing set 95% AUPRC CI for the full LASSO model: 0.881 [0.802, 0.933]


In [18]:
# PCA dataset best AUC
PCA_model = PCA_models['Lasso'].fit(X_train_PCA_new, y_train_new.values.ravel())

c:\Users\Caitlyn\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [19]:
pred_y_PCA = PCA_model.predict_proba(X_test_PCA)

In [20]:
acc_test = roc_auc_score(y_test, pred_y_PCA[:,1])
[lower, upper] = bootstrap(y_test.values.ravel(), pred_y_PCA[:,1], 1000)
print("Testing set 95%% AUC CI for the PCA LASSO model: %5.3f [%5.3f, %5.3f]" % (acc_test, lower, upper))

Testing set 95% AUC CI for the PCA LASSO model: 0.901 [0.845, 0.949]


In [21]:
# Precision-Recall
precision, recall, thresholds = precision_recall_curve(y_test.values.ravel(), pred_y_PCA[:,1])
# AUPRC
auc_precision_recall = auc(recall, precision)
[lower, upper] = bootstrap_pr(y_test.values.ravel(), pred_y_PCA[:,1], 1000)
print()
print("Testing set 95%% AUPRC CI for the PCA LASSO model: %5.3f [%5.3f, %5.3f]" % (auc_precision_recall, lower, upper))


Testing set 95% AUPRC CI for the PCA LASSO model: 0.923 [0.876, 0.964]


In [22]:
# Export Data
pd.DataFrame(pred_y_PCA[:,1]).to_csv("../Predictions/LASSO_PCA_Pred.csv", index = False, header=None)
pd.DataFrame(pred_y[:,1]).to_csv("../Predictions/LASSO_Pred.csv", index = False, header=None)
beta_vals.to_csv("../Outputs/Beta_Coeffs.csv", index = False)